# World Kill Network

Often, to get ahead in life, someone else must die. Characters will kill other characters to secure land for themselves or those they support, they will kill rivals out of spite or allow prisoners to die in the dungeons. The kill network joins dyanasties where a member of one dynasty has killed a member of the other. It is much the same as the marraige netowrk but the edge list consists of killers and their victims instead of characters and their spouses. First I extract the characters with the highest kill count.

In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

### Groupby Killers

In [2]:
client = MongoClient()
characters = client.ck2.characters

In [3]:
pipeline = [ 
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
    "$group": { 
        "_id": "$killer_data._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_data.bn"}
    }
    },
    {"$sort" : {"kill_total" : -1 } },
    {"$limit" : 20}
] 

In [4]:
karl = characters.aggregate(pipeline)

In [5]:
for kill in karl:
    print(kill)

{'_id': 852282, 'kill_total': 42, 'name': 'Bilge'}
{'_id': 840689, 'kill_total': 36, 'name': 'Jochi'}
{'_id': 1037012, 'kill_total': 26, 'name': 'Christophoros'}
{'_id': 756081, 'kill_total': 24, 'name': 'Virmantas'}
{'_id': 562370, 'kill_total': 22, 'name': 'Manassès'}
{'_id': 984359, 'kill_total': 22, 'name': 'Ghalib'}
{'_id': 563001, 'kill_total': 22, 'name': 'Nasraddin'}
{'_id': 558295, 'kill_total': 20, 'name': 'Megistos'}
{'_id': 945499, 'kill_total': 20, 'name': 'Khaidu'}
{'_id': 770755, 'kill_total': 18, 'name': 'Çilbu'}
{'_id': 832705, 'kill_total': 17, 'name': 'Timotheos'}
{'_id': 976146, 'kill_total': 16, 'name': 'Alexandros'}
{'_id': 639606, 'kill_total': 16, 'name': 'Sabuktigin'}
{'_id': 655186, 'kill_total': 16, 'name': 'Madanapala'}
{'_id': 583469, 'kill_total': 16, 'name': 'Najib'}
{'_id': 915974, 'kill_total': 16, 'name': 'Faruk'}
{'_id': 712992, 'kill_total': 15, 'name': 'Theodotos'}
{'_id': 895054, 'kill_total': 15, 'name': 'Vihangpal'}
{'_id': 878688, 'kill_total': 

Of note is Christophoros. He is absent from the below figures because he is lowborn and has no dynasty and is excluded from the aggregation when the lookup tries to join on dynasty.

## Killer Details

In [6]:
pipeline = [ 
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {
    "$group": { 
        "_id": "$killer_data._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_data.bn"},
        "dyn" : {"$first" : "$killer_dyn.name"},
        "c_d" : {"$first" : "$killer_data.c_d"}
    }
    },
    {
        "$project" : { "_id" : 1, "kill_total" : "$kill_total", "name" : {"$concat" : ["$name", " ", "$dyn"]}, "d_d" : "$d_d"}
    },
    {"$sort" : {"kill_total" : -1 } },
    {"$limit" : 20}
] 

In [7]:
karl = characters.aggregate(pipeline)

In [8]:
for kill in karl:
    print(kill)

{'_id': 852282, 'kill_total': 42, 'name': 'Bilge Jochid'}
{'_id': 840689, 'kill_total': 36, 'name': 'Jochi Jochid'}
{'_id': 756081, 'kill_total': 24, 'name': 'Virmantas Saaremaa'}
{'_id': 563001, 'kill_total': 22, 'name': 'Nasraddin Nasraddin'}
{'_id': 562370, 'kill_total': 22, 'name': 'Manassès Karling'}
{'_id': 984359, 'kill_total': 22, 'name': 'Ghalib Ghalibid'}
{'_id': 558295, 'kill_total': 20, 'name': 'Megistos Isauros'}
{'_id': 945499, 'kill_total': 20, 'name': 'Khaidu Olkhunut'}
{'_id': 770755, 'kill_total': 18, 'name': 'Çilbu Uzur'}
{'_id': 832705, 'kill_total': 17, 'name': 'Timotheos Kritopolos'}
{'_id': 639606, 'kill_total': 16, 'name': 'Sabuktigin Sabuktigin'}
{'_id': 583469, 'kill_total': 16, 'name': 'Najib Amrubid'}
{'_id': 915974, 'kill_total': 16, 'name': 'Faruk Hafizid'}
{'_id': 655186, 'kill_total': 16, 'name': 'Madanapala Ayudha'}
{'_id': 976146, 'kill_total': 16, 'name': 'Alexandros Loukanis'}
{'_id': 613722, 'kill_total': 15, 'name': 'Mansur Kufahyid'}
{'_id': 71299

Father and son team Jochi and Bilge Jochid, Emperors of the Mongol Empire, allowed 78 people to die in their dungeons. King Manassès Karling of Germany burned about 18 people at the stake and poisoned a few more for good measure.

# Worst of the Worst

On Reddit /u/Brock_Lobster4445 correctly pointed out 'One important category was left out, and that's "Most child kills"'. I knew this would be a bit tricky because pymongo, datetimes in general and dattimes before the 1600's don't go well together. By pulling all characters who died out into a pandas dataframe I was able to subtract their birth date from their death date to get their age in days and then filter by those who were under 5840 days (16 years) and under 1460 days (4 years) when they were killed...

In [3]:
pipeline = [
    {
        "$match" : {"killer" : {"$exists" : True}}
    },
    {
        "$project" : {"_id" : 1, "bn" : 1, "b_d" : 1, "d_d" : 1}
    }    
]

In [4]:
anikan = characters.aggregate(pipeline)
dead_list = pd.DataFrame(list(anikan))
dead_list["Age"] = (dead_list["d_d"] - dead_list["b_d"]).astype('timedelta64[D]')

#Filter on those who were were under 16 years of age (16 * 365 days)
child_list = dead_list[dead_list["Age"] <= 5840.0]
child_list_ids = [int(i) for i in child_list["_id"]]

In [5]:
pipeline = [ 
    {
        "$match" : {"_id" : {"$in" : child_list_ids}}
    },
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {
    "$group": { 
        "_id": "$killer_data._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_data.bn"},
        "dyn" : {"$first" : "$killer_dyn.name"},
        "d_d" : {"$first" : "$killer_data.d_d"}
    }
    },
    {
        "$project" : { "_id" : 1, "kill_total" : "$kill_total", "name" : {"$concat" : ["$name", " ", "$dyn"]}, "d_d" : "$d_d"}
    },
    {"$sort" : {"kill_total" : -1 } },
    {"$limit" : 20}
] 

In [6]:
for anikan in characters.aggregate(pipeline):
    print(anikan)

{'_id': 883631, 'kill_total': 7, 'name': 'Kuno Ydulfing', 'd_d': datetime.datetime(1299, 8, 8, 0, 0)}
{'_id': 639606, 'kill_total': 7, 'name': 'Sabuktigin Sabuktigin', 'd_d': datetime.datetime(992, 9, 3, 0, 0)}
{'_id': 976146, 'kill_total': 5, 'name': 'Alexandros Loukanis', 'd_d': datetime.datetime(1390, 10, 11, 0, 0)}
{'_id': 765739, 'kill_total': 4, 'name': 'Wayngachi Wayngachi', 'd_d': datetime.datetime(1167, 6, 24, 0, 0)}
{'_id': 862137, 'kill_total': 4, 'name': 'Vasudev Vasudevid', 'd_d': datetime.datetime(1255, 2, 1, 0, 0)}
{'_id': 708401, 'kill_total': 3, 'name': 'Halil Abbasid', 'd_d': datetime.datetime(1104, 8, 2, 0, 0)}
{'_id': 761879, 'kill_total': 3, 'name': 'Hanifa Jaleelid', 'd_d': datetime.datetime(1192, 3, 28, 0, 0)}
{'_id': 937090, 'kill_total': 3, 'name': 'Konstantia Kameniatis', 'd_d': datetime.datetime(1401, 6, 15, 0, 0)}
{'_id': 656924, 'kill_total': 3, 'name': 'Lyaqut Mezwarid', 'd_d': datetime.datetime(1061, 3, 17, 0, 0)}
{'_id': 801821, 'kill_total': 3, 'name': 

Kuno Ydulfing held was the Count of Bereg after taking it in a revolt in 1270 until he was overthrown in a Holy War in 1275. Sabuktigin "The Conqueror" conquered Kabul, Bamiyan, Lahur amoungst others in a Holy War. Korybutas Penikis was King of Lithuania and Ali Arabid was the Emperor of Spain. The skulls surrounding their thrones are somewhat smaller than those around the thrones of other rulers.

In [7]:
baby_list = dead_list[dead_list["Age"] <= 1460.0]

In [8]:
baby_list_ids = [int(i) for i in baby_list["_id"]]

In [9]:
pipeline = [ 
    {
        "$match" : {"_id" : {"$in" : baby_list_ids}}
    },
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {
    "$group": { 
        "_id": "$killer_data._id", 
        "kill_total": { "$sum": 1 },
        "name" : {"$first" : "$killer_data.bn"},
        "dyn" : {"$first" : "$killer_dyn.name"},
        "d_d" : {"$first" : "$killer_data.d_d"}
    }
    },
    {
        "$project" : { "_id" : 1, "kill_total" : "$kill_total", "name" : {"$concat" : ["$name", " ", "$dyn"]}, "d_d" : "$d_d"}
    },
    {"$sort" : {"kill_total" : -1 } },
    {"$limit" : 20}
] 

In [10]:
for anikan in characters.aggregate(pipeline):
    print(anikan)

{'_id': 782314, 'kill_total': 3, 'name': 'Reshawna Mukhtarid', 'd_d': datetime.datetime(1203, 4, 23, 0, 0)}
{'_id': 639606, 'kill_total': 3, 'name': 'Sabuktigin Sabuktigin', 'd_d': datetime.datetime(992, 9, 3, 0, 0)}
{'_id': 846687, 'kill_total': 2, 'name': 'Hemavati Sripathid', 'd_d': datetime.datetime(1258, 7, 24, 0, 0)}
{'_id': 870131, 'kill_total': 2, 'name': 'Khuterkin Uzur', 'd_d': datetime.datetime(1305, 10, 4, 0, 0)}
{'_id': 749227, 'kill_total': 2, 'name': 'Damayanti Surajpalid', 'd_d': datetime.datetime(1163, 2, 19, 0, 0)}
{'_id': 668380, 'kill_total': 2, 'name': 'Taneen Jalilid', 'd_d': datetime.datetime(1048, 1, 21, 0, 0)}
{'_id': 668248, 'kill_total': 2, 'name': 'Mahdi Dofharid', 'd_d': datetime.datetime(1063, 8, 1, 0, 0)}
{'_id': 761879, 'kill_total': 2, 'name': 'Hanifa Jaleelid', 'd_d': datetime.datetime(1192, 3, 28, 0, 0)}
{'_id': 643044, 'kill_total': 2, 'name': 'Eustathios Rangabes', 'd_d': datetime.datetime(992, 12, 25, 0, 0)}
{'_id': 784821, 'kill_total': 2, 'name':

And then there is this bunch. Reshawna was one of the five wives of the Count of Kuwait and was mother to 3 of his children, including his successor. Sabuktigin makes an apperence here as well, 3 of 7 children he killed were under 4 years of age. Khuterkin "The Lionheart" Uzur was King of Khotan. My countryman Lugáed Ua Flannchaid crossed the Irish Sea to conquer Powys in a revolt, or maybe to escape his dark, baby murdering past? Bad luck follewed him also; his cousin Ernán "The Drunkard" Ua Cleirigh pushed him off a balcony and inherited his title. Glenn "The Wise" ruled as King of Scotland and Greece, marking the Celtic lands as a dangerous place to be a baby.

## Kill Network

In [9]:
pipeline = [ 
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"        
    },    
    {
        "$lookup" :
        {
            "from" : "characters",
            "localField" : "killer",
            "foreignField" : "_id",
            "as" : "killer_data"
        }
    },
    {
        "$unwind" : "$killer_data"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "killer_data.dnt",
            "foreignField" : "_id",
            "as" : "killer_dyn"
        }
    },
    {
        "$unwind" : "$killer_dyn"        
    },
    {"$project" : {"name" : "$bn", "dynasty" : "$dynasty._id", "killer" : "$killer_data.bn", "killer_dynasty" : "$killer_dyn._id"}}
]

In [10]:
karl = characters.aggregate(pipeline)

In [11]:
kill_df = pd.DataFrame(list(karl))

## Get all Dynasties involved

In [12]:
total_dyns = set(kill_df['dynasty'].unique())
total_dyns = total_dyns.union(set(kill_df['killer_dynasty'].unique()))
total_dyns_as_ints = [int(i) for i in list(total_dyns)]

In [13]:
dynasties = client.ck2.dynasties

pipeline = [    
    {
        "$match" : {"_id" : {"$in" : total_dyns_as_ints}}
    },
    {
        "$project" : {"name" : 1, "religion" : 1, "culture" : 1}
    },
    {
        "$sort" : {"name" : 1}
    }
]

In [14]:
dyns = dynasties.aggregate(pipeline)

# Build a Network Graph

In [15]:
import networkx as nx
import matplotlib.pyplot as plt

In [16]:
G = nx.Graph()

for dyn in dyns:
    if "name" in dyn and "religion" in dyn and "culture" in dyn: #needs to be added if building the full graph, a family with no name gets in somehow
        G.add_node(dyn["_id"], name = dyn['name'], culture = dyn['culture'], religion = dyn['religion'])

In [17]:
#complete_set = set()

for i in range(len(kill_df)):
    if G.has_edge(kill_df.loc[i, "killer_dynasty"], kill_df.loc[i, "dynasty"]):
        G.edge[kill_df.loc[i, "killer_dynasty"]][kill_df.loc[i, "dynasty"]]["weight"] +=1
    else:
        G.add_edge(kill_df.loc[i, "killer_dynasty"], kill_df.loc[i, "dynasty"], weight = 1)

G.remove_nodes_from(nx.isolates(G)) #drop unconnected nodes       

In [18]:
nx.write_graphml(max(nx.connected_component_subgraphs(G), key=len), "ck2-World-Kill-Network.graphml")

The graphml file in the above was opened in Gephi and the the picture below was generated. The nodes are colored by religion. The graph shares many similarities with the World Marriage Network in notebook 5. India is again in the top right, isolated from much of the rest of the world. Europe is on the bottom left. Orange is Catholic, blue is Islamic and the pink at the top is Buddist. Orthodox Greece is to the left, just above Europe in purple, surrounded by many of the pagan religions.

It would make sense for this graph to be directed, with an arrow pointing from the killers dynasty to the victims dynasty. The code in the cell above this allows me to pull out the largest subgraph in the network but fails if using a DiGraph. For an easy life I deceided to leave it undirected.

In [19]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://www.anquantarbuile.com/static/images/ck2/WorldKillNetworkReligion.png")

# Centrality Measures

In [20]:
# Return stats of graph (degree, centrality etc)
def get_graph_stats(graph, by_col = ''):   
    degree = pd.DataFrame.from_dict(graph.degree(graph), orient = 'index').reset_index()
    degree.rename(columns={'index': 'Name', 0: 'Degree'}, inplace=True)
    
    degree_cent = pd.DataFrame.from_dict(nx.degree_centrality(graph), orient = 'index').reset_index()
    degree_cent.rename(columns={'index': 'Name', 0: 'Deg Cent'}, inplace=True)
    
    stats_df = pd.merge(degree, degree_cent, on = ['Name', 'Name'])
    
    close_cent = pd.DataFrame.from_dict(nx.closeness_centrality(graph), orient = 'index').reset_index()
    close_cent.rename(columns={'index': 'Name', 0: 'Close Cent'}, inplace=True)
    
    stats_df = pd.merge(stats_df, close_cent, on = ['Name', 'Name'])
    
    betw_cent = pd.DataFrame.from_dict(nx.betweenness_centrality(graph), orient = 'index').reset_index()
    betw_cent.rename(columns={'index': 'Name', 0: 'Betw Cent'}, inplace=True)
    
    stats_df = pd.merge(stats_df, betw_cent, on = ['Name', 'Name'])
    
    eigenvector = pd.DataFrame.from_dict(nx.eigenvector_centrality(graph), orient = 'index').reset_index()
    eigenvector.rename(columns={'index': 'Name', 0: 'Eigenvector'}, inplace=True)
    
    stats_df = pd.merge(stats_df, eigenvector, on = ['Name', 'Name'])
    
    pagerank = pd.DataFrame.from_dict(nx.pagerank(graph), orient = 'index').reset_index()
    pagerank.rename(columns={'index': 'Name', 0: 'PageRank'}, inplace=True)
    
    stats_df = pd.merge(stats_df, pagerank, on = ['Name', 'Name'])
    
    if by_col != '':
        stats_df = stats_df.sort_values(by = by_col, ascending = False).reset_index(drop = True)
    
    return stats_df


In [21]:
pipeline = [ 
    { 
        "$project" : {"_id" : "$_id", "name" : "$name", "culture" : "$culture", "religion" : "$religion"}
    }
]

In [22]:
dynasties = client.ck2.dynasties

In [23]:
dyn_list = dynasties.aggregate(pipeline)
dyn_df = pd.DataFrame(list(dyn_list))

In [24]:
stats = get_graph_stats(G)

In [25]:
comb_stats = dyn_df.merge(stats, left_on='_id', right_on='Name', how='outer')
comb_stats = comb_stats.dropna(axis=0, how='any')
comb_stats = comb_stats.drop(["Name"], axis = 1)

In [26]:
comb_stats.sort_values(by = 'PageRank', ascending = False).head(15)

,_id,culture,name,religion,Degree,Deg Cent,Close Cent,Betw Cent,Eigenvector,PageRank
5762,1044301,hindustani,Ayudha,hindu,237.0,0.030692,0.264775,0.105195,0.146453,0.007413
4307,101727,bedouin_arabic,Abbasid,sunni,195.0,0.025253,0.287283,0.081760,0.794772,0.006150
5169,12308,bedouin_arabic,Aslamid,sunni,159.0,0.020591,0.255062,0.034368,0.047255,0.005691
6006,1048001,bedouin_arabic,Muhallabid,sunni,173.0,0.022404,0.271062,0.059265,0.076533,0.005236
1695,8646,greek,Isauros,iconoclast,160.0,0.020720,0.262128,0.036086,0.008396,0.004820
6244,1051100,uyghur,Uzur,manichean,114.0,0.014763,0.249077,0.030490,0.012590,0.004536
1921,9530,lettigallish,Penikis,baltic_pagan,114.0,0.014763,0.245485,0.027876,0.002756,0.004362
610,615,egyptian_arabic,Tabghach,sunni,119.0,0.015411,0.252444,0.030610,0.084705,0.003925
5643,1044053,lombard,Alachisling,catholic,111.0,0.014375,0.267349,0.046155,0.005132,0.003125
5866,1042137,bedouin_arabic,Amrubid,sunni,104.0,0.013468,0.247125,0.015563,0.104250,0.003065


In [27]:
comb_stats.to_csv('CK2-Kill-Network-stats.csv', index=False)